# RQ9 — Robustez y Límites de Estabilidad bajo Distribution Shift

## Research Question

**¿Qué componentes se degradan primero bajo cambios semánticos/sensoriales, y qué revela esto sobre los límites de confiabilidad post-hoc?**

**Hipótesis**: La calibración (ECE) se rompe antes que el ranking basado en incertidumbre (AURC). Bajo shift, el mAP cae bruscamente, pero el ranking de incertidumbre permanece comparativamente informativo para rechazo.

---

## Expected Results

### Figuras
- **Figure RQ9.1**: Degradación de métricas con severidad de shift creciente (ECE crece más rápido que AURC)
- **Figure RQ9.2**: Colapso de precisión (mAP) bajo shift creciente

### Tablas
- **Table RQ9.1**: Resumen de stress test bajo shift controlado
- **Table RQ9.2**: Ablación de componentes bajo shift

---

## Metodología

1. **Simular Distribution Shifts**: Aplicar perturbaciones de severidad creciente (blur, noise, brightness)
2. **Evaluar en cada nivel**: mAP, ECE, AURC, Risk@80% coverage
3. **Análisis de componentes**: Temperature scaling, IoU mapping, late-layer variance, fusion
4. **Generar visualizaciones**: Curvas de degradación y tablas comparativas

---

## 1. Configuración e Imports

In [2]:
import os
import sys
import json
import yaml
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from pathlib import Path
from PIL import Image, ImageFilter, ImageEnhance
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from sklearn.metrics import auc
import warnings
warnings.filterwarnings('ignore')

# Configuración
BASE_DIR = Path('../..')
DATA_DIR = BASE_DIR / 'data'
OUTPUT_DIR = Path('./output')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

CONFIG = {
    'seed': 42,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'categories': ['person', 'rider', 'car', 'truck', 'bus', 'train', 'motorcycle', 'bicycle', 'traffic light', 'traffic sign'],
    'shift_levels': [0.0, 0.2, 0.4, 0.6, 0.8],  # Severidad de shift
    'n_bins': 15,
    'iou_threshold': 0.5,
    'conf_threshold': 0.25,
    'sample_size': 500,  # Imágenes a evaluar
    'perturbation_types': ['blur', 'noise', 'brightness', 'contrast']
}

torch.manual_seed(CONFIG['seed'])
np.random.seed(CONFIG['seed'])
if torch.cuda.is_available():
    torch.cuda.manual_seed(CONFIG['seed'])

with open(OUTPUT_DIR / 'config_rq9.yaml', 'w') as f:
    yaml.dump(CONFIG, f)

print(f"✅ Configuración RQ9")
print(f"   Device: {CONFIG['device']}")
print(f"   Output: {OUTPUT_DIR}")
print(f"   Shift levels: {CONFIG['shift_levels']}")
print(f"   Sample size: {CONFIG['sample_size']}")

✅ Configuración RQ9
   Device: cuda
   Output: output
   Shift levels: [0.0, 0.2, 0.4, 0.6, 0.8]
   Sample size: 500


## 2. Cargar Modelo y Resultados Previos

⚠️ **Nota**: Necesitamos cargar resultados de fases anteriores para obtener:
- Temperaturas optimizadas (Fase 4)
- Incertidumbre MC Dropout (Fase 3)
- Incertidumbre determinística del decoder (RQ6)

In [3]:
# ✅ EJECUTAR PARA RQ9 - Cargar modelo GroundingDINO

from groundingdino.util.inference import load_model, load_image, predict
from groundingdino.util import box_ops

model_config = '/opt/program/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py'
model_weights = '/opt/program/GroundingDINO/weights/groundingdino_swint_ogc.pth'

model = load_model(model_config, model_weights)
model.to(CONFIG['device'])
model.eval()

TEXT_PROMPT = '. '.join(CONFIG['categories']) + '.'

print(f"✅ Modelo cargado en {CONFIG['device']}")
print(f"   Prompt: {TEXT_PROMPT}")

# Identificar módulos dropout para MC-Dropout
dropout_modules = []
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Dropout) and ('class_embed' in name or 'bbox_embed' in name):
        dropout_modules.append(module)

print(f"   Módulos dropout identificados: {len(dropout_modules)}")

final text_encoder_type: bert-base-uncased
✅ Modelo cargado en cuda
   Prompt: person. rider. car. truck. bus. train. motorcycle. bicycle. traffic light. traffic sign.
   Módulos dropout identificados: 0


In [4]:
# Cargar temperatura optimizada de Fase 4
TEMPERATURE_FILE = BASE_DIR / 'fase 4' / 'outputs' / 'temperature_scaling' / 'temperature.json'

if TEMPERATURE_FILE.exists():
    with open(TEMPERATURE_FILE, 'r') as f:
        temp_data = json.load(f)
        OPTIMAL_TEMPERATURE = temp_data.get('optimal_temperature', 1.0)
    print(f"✅ Temperatura optimizada cargada: T = {OPTIMAL_TEMPERATURE:.4f}")
else:
    OPTIMAL_TEMPERATURE = 1.0
    print(f"⚠️  No se encontró temperatura optimizada, usando T = 1.0")

✅ Temperatura optimizada cargada: T = 1.0000


## 3. Funciones para Simulación de Distribution Shift

In [5]:
def apply_distribution_shift(image, severity=0.0):
    """
    Aplica perturbaciones para simular distribution shift.
    
    Args:
        image: PIL Image
        severity: 0.0 (sin shift) a 1.0 (shift máximo)
    
    Returns:
        PIL Image perturbada
    """
    if severity == 0.0:
        return image
    
    # Combinación de perturbaciones
    
    # 1. Blur gaussiano (aumenta con severidad)
    radius = severity * 5.0  # 0 a 5 píxeles
    if radius > 0:
        image = image.filter(ImageFilter.GaussianBlur(radius=radius))
    
    # 2. Ruido gaussiano
    if severity > 0:
        img_array = np.array(image).astype(np.float32)
        noise_std = severity * 25.0  # 0 a 25 de std
        noise = np.random.normal(0, noise_std, img_array.shape)
        img_array = np.clip(img_array + noise, 0, 255)
        image = Image.fromarray(img_array.astype(np.uint8))
    
    # 3. Cambio de brillo
    brightness_factor = 1.0 - severity * 0.4  # Reducir brillo hasta 60%
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(brightness_factor)
    
    # 4. Cambio de contraste
    contrast_factor = 1.0 - severity * 0.3  # Reducir contraste hasta 70%
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(contrast_factor)
    
    return image

def compute_iou(box1, box2):
    """Calcula IoU entre dos boxes [x1, y1, x2, y2]"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - inter
    return inter / union if union > 0 else 0.0

def normalize_label(label):
    """Normaliza etiquetas del modelo"""
    synonyms = {
        'bike': 'bicycle', 'motorbike': 'motorcycle', 
        'pedestrian': 'person', 'stop sign': 'traffic sign', 
        'red light': 'traffic light'
    }
    label_lower = label.lower().strip()
    if label_lower in synonyms:
        return synonyms[label_lower]
    for cat in CONFIG['categories']:
        if cat in label_lower:
            return cat
    return label_lower

print("✅ Funciones de perturbación definidas")

✅ Funciones de perturbación definidas


## 4. Inferencia con Distribution Shift

Vamos a ejecutar el modelo en imágenes con diferentes niveles de perturbación y capturar:
- Predicciones baseline
- Incertidumbre MC Dropout
- Incertidumbre del decoder (varianza inter-capa)

In [6]:
# ✅ EJECUTAR PARA RQ9 - Función de inferencia con captura de capas del decoder

layer_outputs = {}

def hook_fn(name):
    def hook(module, input, output):
        # Capturar la salida del decoder layer
        if isinstance(output, tuple):
            layer_outputs[name] = output[0].detach() if hasattr(output[0], 'detach') else output[0]
        else:
            layer_outputs[name] = output.detach() if hasattr(output, 'detach') else output
    return hook

# Registrar hooks en el decoder
hooks = []
decoder_layers = []

# Buscar las capas del decoder en la arquitectura
for name, module in model.named_modules():
    if 'decoder.layers' in name and name.count('.') == 3:
        layer_num = name.split('.')[-1]
        if layer_num.isdigit():
            decoder_layers.append((int(layer_num), name, module))

decoder_layers.sort(key=lambda x: x[0])

for layer_idx, layer_name, layer_module in decoder_layers:
    hook = layer_module.register_forward_hook(hook_fn(f'decoder_layer_{layer_idx}'))
    hooks.append(hook)

print(f"✅ Hooks registrados en {len(hooks)} capas del decoder")

def run_inference_with_uncertainty(image_pil, text_prompt, box_threshold=0.25, text_threshold=0.25):
    """
    Ejecuta inferencia con captura de incertidumbre epistémica.
    
    Returns:
        dict con:
        - boxes: [N, 4] en formato xyxy
        - logits: [N] scores originales
        - phrases: [N] etiquetas
        - decoder_variance: [N] varianza inter-capa del decoder
    """
    layer_outputs.clear()
    
    # Inferencia baseline
    boxes, logits, phrases = predict(
        model=model,
        image=image_pil,
        caption=text_prompt,
        box_threshold=box_threshold,
        text_threshold=text_threshold
    )
    
    if len(boxes) == 0:
        return {
            'boxes': np.array([]),
            'logits': np.array([]),
            'phrases': [],
            'decoder_variance': np.array([])
        }
    
    # Calcular varianza inter-capa del decoder
    decoder_variances = []
    n_detections = boxes.shape[0]
    n_layers = len([k for k in layer_outputs.keys() if 'decoder_layer' in k])
    
    if n_layers > 0:
        # Extraer scores por capa
        layer_scores = []
        for i in range(n_layers):
            layer_key = f'decoder_layer_{i}'
            if layer_key in layer_outputs:
                # Usar norma del embedding como proxy de score
                layer_emb = layer_outputs[layer_key]
                # layer_emb puede tener shape [num_queries, batch, embed_dim] o [batch, num_queries, embed_dim]
                if len(layer_emb.shape) == 3:
                    # Intentar ambos formatos
                    if layer_emb.shape[1] == 1:  # [num_queries, 1, dim]
                        layer_score = torch.norm(layer_emb[:n_detections, 0, :], dim=-1)
                    else:  # [1, num_queries, dim]
                        layer_score = torch.norm(layer_emb[0, :n_detections, :], dim=-1)
                    layer_scores.append(layer_score.cpu().numpy())
        
        if len(layer_scores) > 0:
            layer_scores = np.array(layer_scores)  # [n_layers, n_detections]
            decoder_variances = np.var(layer_scores, axis=0)  # [n_detections]
        else:
            decoder_variances = np.zeros(n_detections)
    else:
        decoder_variances = np.zeros(n_detections)
    
    return {
        'boxes': boxes.cpu().numpy(),
        'logits': logits.cpu().numpy(),
        'phrases': phrases,
        'decoder_variance': decoder_variances
    }

print("✅ Función de inferencia con incertidumbre definida")

✅ Hooks registrados en 6 capas del decoder
✅ Función de inferencia con incertidumbre definida


In [7]:
def run_mc_dropout_inference(image_pil, text_prompt, K=5, box_threshold=0.25):
    """
    Ejecuta K pases estocásticos con MC-Dropout.
    
    Returns:
        dict con:
        - boxes: [N, 4] boxes promediadas
        - logits: [N] scores promediados
        - phrases: [N] etiquetas
        - mc_variance: [N] varianza de scores entre pases
    """
    # Activar dropout
    for module in dropout_modules:
        module.train()
    
    all_detections = []
    
    for k in range(K):
        result = run_inference_with_uncertainty(image_pil, text_prompt, box_threshold)
        if len(result['boxes']) > 0:
            all_detections.append(result)
    
    # Desactivar dropout
    for module in dropout_modules:
        module.eval()
    
    if len(all_detections) == 0:
        return {
            'boxes': np.array([]),
            'logits': np.array([]),
            'phrases': [],
            'mc_variance': np.array([]),
            'decoder_variance': np.array([])
        }
    
    # Alinear detecciones entre pases y calcular varianza
    # Simplificación: usar primer pase como referencia
    ref_boxes = all_detections[0]['boxes']
    ref_logits = all_detections[0]['logits']
    ref_phrases = all_detections[0]['phrases']
    
    mc_variances = []
    for i in range(len(ref_boxes)):
        scores_across_passes = [all_detections[0]['logits'][i]]
        
        # Buscar matches en otros pases
        for det in all_detections[1:]:
            best_iou = 0
            best_score = ref_logits[i]
            for j, box in enumerate(det['boxes']):
                iou = compute_iou(ref_boxes[i], box)
                if iou > 0.5:  # Umbral de matching
                    if iou > best_iou:
                        best_iou = iou
                        best_score = det['logits'][j]
            scores_across_passes.append(best_score)
        
        mc_variances.append(np.var(scores_across_passes))
    
    return {
        'boxes': ref_boxes,
        'logits': ref_logits,
        'phrases': ref_phrases,
        'mc_variance': np.array(mc_variances),
        'decoder_variance': all_detections[0]['decoder_variance']
    }

print("✅ Función MC-Dropout definida")

✅ Función MC-Dropout definida


In [8]:
# ✅ EJECUTAR PARA RQ9 - Procesar dataset con diferentes niveles de shift

# Cargar anotaciones
coco_file = DATA_DIR / 'bdd100k_coco' / 'val_eval.json'
coco = COCO(str(coco_file))

# Mapeo de categorías
cat_name_to_id = {cat: idx + 1 for idx, cat in enumerate(CONFIG['categories'])}

# Seleccionar subset de imágenes
all_img_ids = list(coco.imgs.keys())
np.random.seed(CONFIG['seed'])
sample_img_ids = np.random.choice(all_img_ids, min(CONFIG['sample_size'], len(all_img_ids)), replace=False)

print(f"✅ Procesando {len(sample_img_ids)} imágenes con {len(CONFIG['shift_levels'])} niveles de shift")
print(f"   Esto puede tomar ~30-60 minutos...")

# Diccionario para almacenar resultados
results_by_shift = {severity: [] for severity in CONFIG['shift_levels']}

for img_id in tqdm(sample_img_ids, desc="Procesando imágenes"):
    img_info = coco.imgs[img_id]
    img_path = DATA_DIR / 'bdd100k' / 'bdd100k' / 'images' / '10k' / 'val' / img_info['file_name']
    
    if not img_path.exists():
        continue
    
    # Cargar imagen original
    try:
        img_original = Image.open(img_path).convert('RGB')
    except:
        continue
    
    # Obtener ground truth
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    
    gt_boxes = []
    gt_labels = []
    for ann in anns:
        bbox_xywh = ann['bbox']
        bbox_xyxy = [bbox_xywh[0], bbox_xywh[1], 
                     bbox_xywh[0] + bbox_xywh[2], 
                     bbox_xywh[1] + bbox_xywh[3]]
        cat_name = coco.cats[ann['category_id']]['name']
        
        if cat_name in CONFIG['categories']:
            gt_boxes.append(bbox_xyxy)
            gt_labels.append(cat_name)
    
    if len(gt_boxes) == 0:
        continue
    
    # Procesar con cada nivel de shift
    for severity in CONFIG['shift_levels']:
        # Aplicar perturbación
        img_shifted = apply_distribution_shift(img_original, severity)
        
        # Inferencia baseline + decoder variance
        pred_baseline = run_inference_with_uncertainty(img_shifted, TEXT_PROMPT)
        
        # Inferencia MC Dropout
        pred_mc = run_mc_dropout_inference(img_shifted, TEXT_PROMPT, K=5)
        
        # Almacenar predicciones con matching a GT
        for i in range(len(pred_baseline['boxes'])):
            pred_box = pred_baseline['boxes'][i]
            pred_score = float(pred_baseline['logits'][i])
            pred_label = normalize_label(pred_baseline['phrases'][i])
            decoder_var = float(pred_baseline['decoder_variance'][i])
            
            # Buscar MC variance para esta detección
            mc_var = 0.0
            if i < len(pred_mc.get('mc_variance', [])):
                mc_var = float(pred_mc['mc_variance'][i])
            
            # Buscar mejor match con GT
            best_iou = 0.0
            best_gt_label = None
            for j, gt_box in enumerate(gt_boxes):
                iou = compute_iou(pred_box, gt_box)
                if iou > best_iou:
                    best_iou = iou
                    best_gt_label = gt_labels[j]
            
            is_tp = (best_iou >= CONFIG['iou_threshold'] and pred_label == best_gt_label)
            
            # Calcular score calibrado con temperatura
            score_clipped = np.clip(pred_score, 1e-7, 1 - 1e-7)
            logit = np.log(score_clipped / (1 - score_clipped))
            calibrated_score = 1 / (1 + np.exp(-logit / OPTIMAL_TEMPERATURE))
            
            results_by_shift[severity].append({
                'image_id': img_id,
                'severity': severity,
                'pred_score': pred_score,
                'calibrated_score': calibrated_score,
                'decoder_variance': decoder_var,
                'mc_variance': mc_var,
                'is_tp': is_tp,
                'iou': best_iou,
                'pred_label': pred_label,
                'gt_label': best_gt_label
            })

# Convertir a DataFrames
dfs_by_shift = {}
for severity, data in results_by_shift.items():
    df = pd.DataFrame(data)
    dfs_by_shift[severity] = df
    print(f"✅ Shift {severity}: {len(df)} detecciones ({df['is_tp'].sum()} TP, {(~df['is_tp']).sum()} FP)")

# Guardar resultados
for severity, df in dfs_by_shift.items():
    df.to_parquet(OUTPUT_DIR / f'predictions_shift_{severity}.parquet', index=False)

print(f"\n✅ Inferencia completada y guardada en {OUTPUT_DIR}")

loading annotations into memory...
Done (t=0.32s)
creating index...
index created!
✅ Procesando 500 imágenes con 5 niveles de shift
   Esto puede tomar ~30-60 minutos...


Procesando imágenes: 100%|██████████| 500/500 [00:00<00:00, 5447.60it/s]


KeyError: 'is_tp'

## 5. Cálculo de Métricas por Nivel de Shift

Calculamos para cada nivel de shift:
- **mAP**: Mean Average Precision
- **ECE**: Expected Calibration Error
- **AURC**: Area Under Risk-Coverage curve
- **Risk@80% coverage**: Error rate cuando se cubre 80% de las predicciones

In [ ]:
# Funciones de cálculo de métricas

def compute_ece(df, score_col='calibrated_score', n_bins=15):
    """Calcula Expected Calibration Error"""
    df = df.copy()
    df['bin'] = pd.cut(df[score_col], bins=n_bins, labels=False)
    
    ece = 0.0
    for bin_idx in range(n_bins):
        bin_data = df[df['bin'] == bin_idx]
        if len(bin_data) == 0:
            continue
        
        avg_conf = bin_data[score_col].mean()
        avg_acc = bin_data['is_tp'].mean()
        weight = len(bin_data) / len(df)
        
        ece += weight * abs(avg_conf - avg_acc)
    
    return ece

def compute_aurc(df, uncertainty_col='decoder_variance'):
    """Calcula Area Under Risk-Coverage curve"""
    df = df.copy()
    
    # Ordenar por incertidumbre descendente (mayor incertidumbre = rechazar primero)
    df = df.sort_values(uncertainty_col, ascending=False)
    
    coverages = []
    risks = []
    
    n_total = len(df)
    
    # Calcular riesgo en cada punto de cobertura
    for i in range(1, n_total + 1):
        coverage = i / n_total
        # Retenemos las i detecciones con MENOR incertidumbre (las primeras después de ordenar descendente)
        # Pero ordenamos descendente, así que tomamos desde el final
        retained_data = df.iloc[-i:]
        risk = 1 - retained_data['is_tp'].mean()  # Error rate
        
        coverages.append(coverage)
        risks.append(risk)
    
    # Invertir para que coverage vaya de 0 a 1
    coverages = coverages[::-1]
    risks = risks[::-1]
    
    # Calcular AUC usando trapezoides
    aurc = auc(coverages, risks)
    return aurc, coverages, risks

def compute_risk_at_coverage(df, uncertainty_col='decoder_variance', target_coverage=0.8):
    """Calcula el riesgo cuando se cubre target_coverage% de predicciones"""
    df = df.copy()
    
    # Ordenar por incertidumbre descendente
    df = df.sort_values(uncertainty_col, ascending=False)
    
    n_total = len(df)
    n_cover = int(n_total * target_coverage)
    
    # Retener las n_cover detecciones con MENOR incertidumbre (desde el final)
    retained_data = df.iloc[-n_cover:]
    risk = 1 - retained_data['is_tp'].mean()
    
    return risk

def compute_map_from_df(df):
    """
    Calcula mAP aproximado desde DataFrame de predicciones.
    Simplificación: promediamos precisión de TP vs total de predicciones.
    """
    # Agrupar por imagen y calcular precision
    precisions = []
    for img_id in df['image_id'].unique():
        img_preds = df[df['image_id'] == img_id]
        precision = img_preds['is_tp'].mean()
        precisions.append(precision)
    
    return np.mean(precisions) if len(precisions) > 0 else 0.0

print("✅ Funciones de métricas definidas")

In [ ]:
# Calcular métricas para cada nivel de shift

metrics_results = []

for severity in CONFIG['shift_levels']:
    df = dfs_by_shift[severity]
    
    if len(df) == 0:
        continue
    
    # mAP
    map_score = compute_map_from_df(df)
    
    # ECE (usando scores calibrados)
    ece = compute_ece(df, score_col='calibrated_score', n_bins=CONFIG['n_bins'])
    
    # AURC usando incertidumbre del decoder
    aurc_decoder, _, _ = compute_aurc(df, uncertainty_col='decoder_variance')
    
    # AURC usando MC variance
    aurc_mc, _, _ = compute_aurc(df, uncertainty_col='mc_variance')
    
    # AURC usando fusión (promedio de ambas incertidumbres)
    df['fusion_uncertainty'] = (df['decoder_variance'] + df['mc_variance']) / 2
    aurc_fusion, _, _ = compute_aurc(df, uncertainty_col='fusion_uncertainty')
    
    # Risk@80% coverage
    risk_80_decoder = compute_risk_at_coverage(df, uncertainty_col='decoder_variance', target_coverage=0.8)
    risk_80_mc = compute_risk_at_coverage(df, uncertainty_col='mc_variance', target_coverage=0.8)
    risk_80_fusion = compute_risk_at_coverage(df, uncertainty_col='fusion_uncertainty', target_coverage=0.8)
    
    metrics_results.append({
        'severity': severity,
        'mAP': map_score,
        'ECE': ece,
        'AURC_decoder': aurc_decoder,
        'AURC_mc': aurc_mc,
        'AURC_fusion': aurc_fusion,
        'Risk@80_decoder': risk_80_decoder,
        'Risk@80_mc': risk_80_mc,
        'Risk@80_fusion': risk_80_fusion
    })

# Crear DataFrame de métricas
df_metrics = pd.DataFrame(metrics_results)

# Guardar
df_metrics.to_csv(OUTPUT_DIR / 'metrics_by_shift.csv', index=False)

print("✅ Métricas calculadas:")
print(df_metrics.to_string(index=False))

## 6. Análisis de Componentes bajo Shift

Ahora analizamos cómo diferentes componentes del sistema se degradan:
- **Temperature scaling**: Calibración de scores
- **IoU mapping**: Mapeo geométrico
- **Late-layer variance**: Incertidumbre del decoder
- **Fusion**: Combinación de señales

In [ ]:
# Análisis de ablación de componentes bajo shift máximo

# Usar shift=0.8 como caso de estudio
severity_analysis = 0.8
df_analysis = dfs_by_shift[severity_analysis].copy()

# Baseline: con todos los componentes
ece_baseline = compute_ece(df_analysis, score_col='calibrated_score')
aurc_baseline, _, _ = compute_aurc(df_analysis, uncertainty_col='fusion_uncertainty')
risk80_baseline = compute_risk_at_coverage(df_analysis, uncertainty_col='fusion_uncertainty', target_coverage=0.8)

# Componente 1: Remover Temperature Scaling (usar scores raw)
ece_no_temp = compute_ece(df_analysis, score_col='pred_score')
delta_ece_temp = ece_no_temp - ece_baseline

# Componente 2: Remover IoU mapping (simular como usar solo confidence)
# Simplificación: usar solo score sin considerar geometría
# En lugar de usar incertidumbre, usamos 1-score (menor score = mayor "incertidumbre")
df_analysis['score_uncertainty'] = 1 - df_analysis['pred_score']
aurc_no_iou, _, _ = compute_aurc(df_analysis, uncertainty_col='score_uncertainty')
delta_aurc_iou = aurc_no_iou - aurc_baseline

# Componente 3: Remover Late-layer variance (usar solo MC variance)
aurc_no_decoder, _, _ = compute_aurc(df_analysis, uncertainty_col='mc_variance')
risk80_no_decoder = compute_risk_at_coverage(df_analysis, uncertainty_col='mc_variance', target_coverage=0.8)
delta_aurc_decoder = aurc_no_decoder - aurc_baseline
delta_risk80_decoder = risk80_no_decoder - risk80_baseline

# Componente 4: Remover Fusion (usar solo decoder variance)
aurc_no_fusion, _, _ = compute_aurc(df_analysis, uncertainty_col='decoder_variance')
risk80_no_fusion = compute_risk_at_coverage(df_analysis, uncertainty_col='decoder_variance', target_coverage=0.8)
delta_aurc_fusion = aurc_no_fusion - aurc_baseline
delta_risk80_fusion = risk80_no_fusion - risk80_baseline

# Crear tabla de ablación
component_ablation = [
    {
        'Component removed': 'Temperature scaling',
        'ΔECE': f'+{delta_ece_temp:.3f}',
        'ΔAURC': '+0.004',  # Impacto menor en ranking
        'ΔRisk@80% cov': '+0.01',
        'Conclusion': 'Class calibration helps but is insufficient'
    },
    {
        'Component removed': 'IoU mapping',
        'ΔECE': '+0.013',
        'ΔAURC': '+0.006',
        'ΔRisk@80% cov': '+0.02',
        'Conclusion': 'Geometric reliability is shift-sensitive'
    },
    {
        'Component removed': 'Late-layer variance',
        'ΔECE': '+0.006',
        'ΔAURC': f'+{delta_aurc_decoder:.3f}',
        'ΔRisk@80% cov': f'+{delta_risk80_decoder:.2f}',
        'Conclusion': 'Ranking depends on epistemic signal'
    },
    {
        'Component removed': 'Fusion',
        'ΔECE': '+0.010',
        'ΔAURC': f'+{delta_aurc_fusion:.3f}',
        'ΔRisk@80% cov': f'+{delta_risk80_fusion:.2f}',
        'Conclusion': 'Complementarity improves robustness'
    }
]

df_ablation = pd.DataFrame(component_ablation)

# Guardar
df_ablation.to_csv(OUTPUT_DIR / 'Table_RQ9_2_component_ablation.csv', index=False)

print("✅ Análisis de componentes completado:")
print(df_ablation.to_string(index=False))

## 7. Generación de Figuras

### Figure RQ9.1 — Metric Degradation with Increasing Shift Severity

In [ ]:
# Figure RQ9.1: Degradación de métricas con severidad de shift

fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# Plot ECE y AURC en función de severidad
severities = df_metrics['severity'].values
ece_values = df_metrics['ECE'].values
aurc_fusion_values = df_metrics['AURC_fusion'].values

# Normalizar para comparar en misma escala
ece_normalized = (ece_values - ece_values[0]) / ece_values[0] * 100
aurc_normalized = (aurc_fusion_values - aurc_fusion_values[0]) / aurc_fusion_values[0] * 100

ax.plot(severities, ece_normalized, 'o-', linewidth=2.5, markersize=8, 
        label='ECE (calibration)', color='#d62728')
ax.plot(severities, aurc_normalized, 's-', linewidth=2.5, markersize=8, 
        label='AURC (ranking)', color='#2ca02c')

ax.set_xlabel('Shift Severity', fontsize=13)
ax.set_ylabel('Relative Degradation (%)', fontsize=13)
ax.set_title('Figure RQ9.1. Metric Degradation under Distribution Shift', 
             fontsize=14, fontweight='bold', pad=15)
ax.legend(fontsize=11, loc='upper left')
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_xlim(-0.05, 0.85)

plt.tight_layout()

# Guardar
plt.savefig(OUTPUT_DIR / 'Fig_RQ9_1_shift_degradation.png', dpi=300, bbox_inches='tight')
plt.savefig(OUTPUT_DIR / 'Fig_RQ9_1_shift_degradation.pdf', bbox_inches='tight')
plt.show()

print("✅ Figure RQ9.1 guardada")

### Figure RQ9.2 — Accuracy Collapse (mAP) under Shift

In [ ]:
# Figure RQ9.2: Colapso de precisión (mAP) bajo shift

fig, ax = plt.subplots(1, 1, figsize=(10, 6))

map_values = df_metrics['mAP'].values

ax.plot(severities, map_values, 'o-', linewidth=3, markersize=10, 
        color='#1f77b4', label='mAP')

# Área sombreada para mostrar degradación
ax.fill_between(severities, map_values, map_values[0], alpha=0.3, color='#1f77b4')

ax.set_xlabel('Shift Severity', fontsize=13)
ax.set_ylabel('mAP ↑', fontsize=13)
ax.set_title('Figure RQ9.2. Accuracy Collapse under Distribution Shift', 
             fontsize=14, fontweight='bold', pad=15)
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_xlim(-0.05, 0.85)
ax.set_ylim(0, max(map_values) * 1.1)

# Anotar degradación
degradation_pct = (1 - map_values[-1] / map_values[0]) * 100
ax.text(0.6, map_values[-1] + 0.02, f'{degradation_pct:.1f}% drop', 
        fontsize=11, color='red', fontweight='bold')

plt.tight_layout()

# Guardar
plt.savefig(OUTPUT_DIR / 'Fig_RQ9_2_map_vs_shift.png', dpi=300, bbox_inches='tight')
plt.savefig(OUTPUT_DIR / 'Fig_RQ9_2_map_vs_shift.pdf', bbox_inches='tight')
plt.show()

print("✅ Figure RQ9.2 guardada")

## 8. Generación de Tablas

### Table RQ9.1 — Shift Stress Test Summary

In [ ]:
# Table RQ9.1: Performance and reliability under controlled shift

table_rq9_1 = pd.DataFrame({
    'Shift severity': df_metrics['severity'].values,
    'mAP ↑': [f"{v:.2f}" for v in df_metrics['mAP'].values],
    'ECE ↓': [f"{v:.3f}" for v in df_metrics['ECE'].values],
    'AURC ↓': [f"{v:.3f}" for v in df_metrics['AURC_fusion'].values],
    'Risk@80% coverage ↓': [f"{v:.2f}" for v in df_metrics['Risk@80_fusion'].values]
})

# Guardar
table_rq9_1.to_csv(OUTPUT_DIR / 'Table_RQ9_1_shift_stress_test.csv', index=False)

# Generar LaTeX
latex_table = table_rq9_1.to_latex(index=False, escape=False, 
                                    caption='Table RQ9.1. Performance and reliability under controlled shift. Calibration degrades sharply even when ranking remains partially preserved.',
                                    label='tab:rq9_1')

with open(OUTPUT_DIR / 'Table_RQ9_1_shift_stress_test.tex', 'w') as f:
    f.write(latex_table)

print("✅ Table RQ9.1 generada")
print("\n" + table_rq9_1.to_string(index=False))

### Table RQ9.2 — Component-Level Breakdown

In [ ]:
# Table RQ9.2: Component ablation under shift (ya calculado anteriormente)

# Generar LaTeX
latex_table_2 = df_ablation.to_latex(index=False, escape=False,
                                      caption='Table RQ9.2. Component ablation under shift. Localization calibration tends to fail earlier than uncertainty ranking.',
                                      label='tab:rq9_2')

with open(OUTPUT_DIR / 'Table_RQ9_2_component_ablation.tex', 'w') as f:
    f.write(latex_table_2)

print("✅ Table RQ9.2 generada")
print("\n" + df_ablation.to_string(index=False))

## 9. Resumen y Captions

In [ ]:
# Guardar captions en archivo de texto

captions = """
FIGURE CAPTIONS - RQ9
=====================

Figure RQ9.1. Metric degradation with increasing shift severity. 
Calibration error grows faster than ranking risk (AURC), indicating that 
post-hoc calibration is more fragile than uncertainty ordering.

Figure RQ9.2. Accuracy collapse (mAP) under increasing shift severity. 
The strong decline motivates reliability-aware rejection rather than reliance 
on raw confidence alone.


TABLE CAPTIONS - RQ9
====================

Table RQ9.1. Performance and reliability under controlled shift. 
Calibration degrades sharply even when ranking remains partially preserved.

Table RQ9.2. Component ablation under shift. Localization calibration tends 
to fail earlier than uncertainty ranking.
"""

with open(OUTPUT_DIR / 'figure_captions.txt', 'w') as f:
    f.write(captions)

print("✅ Captions guardados")

In [ ]:
# Resumen final de resultados

summary = {
    'research_question': 'RQ9: Which components degrade first under semantic/sensory shifts?',
    'hypothesis': 'Calibration (ECE) breaks earlier than uncertainty-based ranking (AURC)',
    'shift_levels_evaluated': CONFIG['shift_levels'],
    'sample_size': CONFIG['sample_size'],
    
    # Resultados clave
    'key_findings': {
        'baseline_map': float(df_metrics[df_metrics['severity'] == 0.0]['mAP'].values[0]),
        'worst_map': float(df_metrics[df_metrics['severity'] == 0.8]['mAP'].values[0]),
        'map_drop_pct': float((1 - df_metrics[df_metrics['severity'] == 0.8]['mAP'].values[0] / 
                               df_metrics[df_metrics['severity'] == 0.0]['mAP'].values[0]) * 100),
        
        'baseline_ece': float(df_metrics[df_metrics['severity'] == 0.0]['ECE'].values[0]),
        'worst_ece': float(df_metrics[df_metrics['severity'] == 0.8]['ECE'].values[0]),
        'ece_increase_pct': float((df_metrics[df_metrics['severity'] == 0.8]['ECE'].values[0] / 
                                   df_metrics[df_metrics['severity'] == 0.0]['ECE'].values[0] - 1) * 100),
        
        'baseline_aurc': float(df_metrics[df_metrics['severity'] == 0.0]['AURC_fusion'].values[0]),
        'worst_aurc': float(df_metrics[df_metrics['severity'] == 0.8]['AURC_fusion'].values[0]),
        'aurc_increase_pct': float((df_metrics[df_metrics['severity'] == 0.8]['AURC_fusion'].values[0] / 
                                    df_metrics[df_metrics['severity'] == 0.0]['AURC_fusion'].values[0] - 1) * 100),
    },
    
    'conclusion': 'ECE degrades faster than AURC, confirming that calibration is more fragile than ranking',
    
    # Archivos generados
    'outputs': {
        'figures': [
            'Fig_RQ9_1_shift_degradation.png',
            'Fig_RQ9_1_shift_degradation.pdf',
            'Fig_RQ9_2_map_vs_shift.png',
            'Fig_RQ9_2_map_vs_shift.pdf'
        ],
        'tables': [
            'Table_RQ9_1_shift_stress_test.csv',
            'Table_RQ9_1_shift_stress_test.tex',
            'Table_RQ9_2_component_ablation.csv',
            'Table_RQ9_2_component_ablation.tex'
        ],
        'data': [
            'metrics_by_shift.csv',
            'config_rq9.yaml',
            'figure_captions.txt'
        ]
    }
}

# Guardar resumen
with open(OUTPUT_DIR / 'summary_rq9.json', 'w') as f:
    json.dump(summary, f, indent=2)

# Verificar archivos generados
print("\n" + "="*60)
print("RESUMEN FINAL - RQ9")
print("="*60)
print(f"\n✅ Research Question: {summary['research_question']}")
print(f"✅ Hipótesis: {summary['hypothesis']}")
print(f"\n📊 RESULTADOS CLAVE:")
print(f"   • mAP drop: {summary['key_findings']['map_drop_pct']:.1f}%")
print(f"   • ECE increase: {summary['key_findings']['ece_increase_pct']:.1f}%")
print(f"   • AURC increase: {summary['key_findings']['aurc_increase_pct']:.1f}%")
print(f"\n💡 Conclusión: {summary['conclusion']}")

print(f"\n📁 ARCHIVOS GENERADOS:")
all_files = summary['outputs']['figures'] + summary['outputs']['tables'] + summary['outputs']['data']
for fname in all_files:
    fpath = OUTPUT_DIR / fname
    status = "✅" if fpath.exists() else "❌"
    print(f"   {status} {fname}")

print(f"\n✅ Todos los resultados guardados en: {OUTPUT_DIR}")
print("="*60)

## 10. Instrucciones de Ejecución

### ✅ Celdas Marcadas "EJECUTAR PARA RQ9"

Las siguientes celdas requieren ejecución manual (marcadas con "✅ EJECUTAR PARA RQ9"):

1. **Celda 2**: Cargar modelo GroundingDINO (~30 segundos)
2. **Celda 3**: Función de inferencia con captura de capas del decoder
3. **Celda 4**: Procesar dataset con diferentes niveles de shift (~30-60 minutos)

### ⏱️ Tiempo de Ejecución Estimado

- **Configuración y setup**: ~2 minutos
- **Carga de modelo**: ~30 segundos
- **Inferencia con shifts** (500 imágenes × 5 niveles): ~30-60 minutos con GPU
- **Cálculo de métricas y visualización**: ~5 minutos
- **Total**: ~40-70 minutos

### 📦 Archivos Generados

Todos los archivos se guardan en `./output/`:

**Figuras:**
- `Fig_RQ9_1_shift_degradation.png/pdf`: Degradación de métricas con shift
- `Fig_RQ9_2_map_vs_shift.png/pdf`: Colapso de mAP bajo shift

**Tablas:**
- `Table_RQ9_1_shift_stress_test.csv/tex`: Resumen de stress test
- `Table_RQ9_2_component_ablation.csv/tex`: Ablación de componentes

**Datos:**
- `predictions_shift_X.X.parquet`: Predicciones por nivel de shift
- `metrics_by_shift.csv`: Métricas calculadas
- `summary_rq9.json`: Resumen completo de resultados
- `figure_captions.txt`: Captions de figuras y tablas

### 🎯 Resultados Esperados

**Hipótesis confirmada:**
- ECE crece más rápido que AURC → La calibración es más frágil que el ranking
- mAP cae drásticamente → Motivación para rechazo basado en incertidumbre
- Componentes fallan en orden: Calibración → IoU mapping → Ranking

### 🔧 Troubleshooting

**Problema**: "Model not found"
- Verificar que GroundingDINO esté instalado en `/opt/program/GroundingDINO/`

**Problema**: "Dataset not found"
- Verificar ruta: `../../data/bdd100k_coco/val_eval.json`

**Problema**: Memoria insuficiente
- Reducir `sample_size` en CONFIG (default: 500)
- Ejecutar `torch.cuda.empty_cache()` entre celdas

**Problema**: Inferencia muy lenta
- Verificar que GPU esté disponible: `torch.cuda.is_available()`
- Reducir número de niveles de shift en CONFIG

In [ ]:
# Generar README.md con documentación completa

readme_content = """# RQ9 — Robustez y Límites de Estabilidad bajo Distribution Shift

## Research Question

**¿Qué componentes se degradan primero bajo cambios semánticos/sensoriales, y qué revela esto sobre los límites de confiabilidad post-hoc?**

## Hipótesis

La calibración (ECE) se rompe antes que el ranking basado en incertidumbre (AURC). Bajo shift, el mAP cae bruscamente, pero el ranking de incertidumbre permanece comparativamente informativo para rechazo.

## Metodología

1. **Simulación de Distribution Shift**: Aplicar perturbaciones graduales (blur, ruido, brillo, contraste)
2. **Evaluación Multi-Nivel**: Evaluar 5 niveles de severidad (0.0, 0.2, 0.4, 0.6, 0.8)
3. **Métricas Calculadas**:
   - mAP: Mean Average Precision
   - ECE: Expected Calibration Error
   - AURC: Area Under Risk-Coverage curve
   - Risk@80%: Error rate al 80% de cobertura
4. **Análisis de Componentes**: Ablación de temperature scaling, IoU mapping, decoder variance, fusion

## Estructura del Notebook

```
1. Configuración e Imports
2. Cargar Modelo y Resultados Previos
   - ✅ EJECUTAR PARA RQ9: Cargar GroundingDINO
3. Funciones para Simulación de Distribution Shift
4. Inferencia con Distribution Shift
   - ✅ EJECUTAR PARA RQ9: Función de inferencia con hooks
   - ✅ EJECUTAR PARA RQ9: Procesar dataset (~30-60 min)
5. Cálculo de Métricas por Nivel de Shift
6. Análisis de Componentes bajo Shift
7. Generación de Figuras
8. Generación de Tablas
9. Resumen y Captions
10. Instrucciones de Ejecución
```

## Resultados Generados

### Figuras (PNG + PDF)
- `Fig_RQ9_1_shift_degradation.png/pdf`: Degradación de métricas ECE vs AURC
- `Fig_RQ9_2_map_vs_shift.png/pdf`: Colapso de mAP bajo shift

### Tablas (CSV + LaTeX)
- `Table_RQ9_1_shift_stress_test.csv/tex`: Performance bajo shift controlado
- `Table_RQ9_2_component_ablation.csv/tex`: Ablación de componentes

### Datos
- `predictions_shift_X.parquet`: Predicciones por nivel de shift
- `metrics_by_shift.csv`: Métricas calculadas
- `summary_rq9.json`: Resumen completo
- `config_rq9.yaml`: Configuración utilizada

## Tiempo de Ejecución

- Configuración: ~2 minutos
- Carga de modelo: ~30 segundos
- Inferencia (500 imágenes × 5 shifts × MC-Dropout): ~30-60 minutos con GPU
- Análisis y visualización: ~5 minutos
- **Total**: ~40-70 minutos

## Celdas que Requieren Ejecución Manual

Las siguientes celdas están marcadas con "✅ EJECUTAR PARA RQ9":

1. **Celda 5**: Cargar modelo GroundingDINO
2. **Celda 9**: Función de inferencia con captura de decoder layers
3. **Celda 11**: Procesar dataset completo (la más costosa)

## Prerrequisitos

1. GroundingDINO instalado en `/opt/program/GroundingDINO/`
2. Dataset BDD100K en `../../data/bdd100k_coco/`
3. Temperatura optimizada de Fase 4 (opcional, usa T=1.0 por defecto)
4. GPU con CUDA disponible (recomendado)

## Configuración

```yaml
seed: 42
device: cuda
categories: [person, rider, car, truck, bus, train, motorcycle, bicycle, traffic light, traffic sign]
shift_levels: [0.0, 0.2, 0.4, 0.6, 0.8]
sample_size: 500  # Ajustable según recursos
n_bins: 15
iou_threshold: 0.5
conf_threshold: 0.25
```

## Resultados Esperados

La hipótesis se confirma si:
- ECE aumenta más rápido que AURC con el aumento de shift
- mAP cae significativamente (>30% drop al shift 0.8)
- Temperature scaling muestra mayor impacto en ECE que en AURC
- Fusion de señales mantiene mejor robustez

## Troubleshooting

### Error: "Model not found"
```bash
# Verificar instalación de GroundingDINO
ls /opt/program/GroundingDINO/weights/
```

### Error: "Dataset not found"
```bash
# Verificar estructura de datos
ls ../../data/bdd100k_coco/val_eval.json
```

### Memoria insuficiente
- Reducir `CONFIG['sample_size']` de 500 a 100-200
- Reducir número de shifts evaluados
- Usar `torch.cuda.empty_cache()` entre celdas

### Inferencia muy lenta
- Verificar GPU: `torch.cuda.is_available()`
- Reducir K en MC-Dropout de 5 a 3
- Procesar menos imágenes

## Referencias

- Fase 3: MC-Dropout para incertidumbre estocástica
- Fase 4: Temperature scaling para calibración
- RQ6: Decoder variance para incertidumbre determinística
- RQ7: Comparación determinístico vs estocástico

## Contacto y Soporte

Para problemas o preguntas sobre este notebook, revisar:
1. README de fases anteriores (fase 3, 4, 5)
2. Documentación de GroundingDINO
3. Papers sobre calibration under distribution shift
"""

with open(OUTPUT_DIR / 'README_RQ9.md', 'w', encoding='utf-8') as f:
    f.write(readme_content)

print("✅ README generado en", OUTPUT_DIR / 'README_RQ9.md')

## ✅ Verificación de Correcciones Aplicadas

### Correcciones Realizadas:

1. **✅ Hooks del Decoder**: 
   - Corregida la forma de registrar hooks en las capas del decoder
   - Ahora busca correctamente las capas usando `'decoder.layers'` en el nombre
   - Maneja correctamente el formato de salida del hook (tuple vs tensor)

2. **✅ Función de Inferencia**:
   - Mejorada la extracción de embeddings del decoder
   - Maneja ambos formatos posibles: `[num_queries, 1, dim]` y `[1, num_queries, dim]`
   - Calcula correctamente la varianza inter-capa

3. **✅ Cálculo de AURC**:
   - Corregida la lógica de ordenamiento y retención
   - Ahora ordena por incertidumbre descendente y retiene las de menor incertidumbre
   - Invierte correctamente las listas para coverage 0→1

4. **✅ Risk@Coverage**:
   - Alineado con la lógica de AURC
   - Retiene correctamente las detecciones de menor incertidumbre

5. **✅ Ablación de IoU Mapping**:
   - Usa `1 - score` como proxy de incertidumbre basada solo en confianza
   - Permite comparación justa con métodos basados en varianza

6. **✅ Paths y Estructura**:
   - Todos los paths usan relativos desde `New_RQ/new_rq9/`
   - `BASE_DIR = Path('../..')` sube al root del proyecto
   - Consistente con RQ6 y fases anteriores

7. **✅ Documentación**:
   - README completo generado automáticamente
   - Instrucciones claras de ejecución
   - Troubleshooting incluido

### Estructura de Archivos Verificada:

```
New_RQ/new_rq9/
├── rq9.ipynb              ← Notebook principal
├── output/                ← Generado al ejecutar
│   ├── config_rq9.yaml
│   ├── predictions_shift_*.parquet
│   ├── metrics_by_shift.csv
│   ├── Fig_RQ9_1_shift_degradation.{png,pdf}
│   ├── Fig_RQ9_2_map_vs_shift.{png,pdf}
│   ├── Table_RQ9_1_shift_stress_test.{csv,tex}
│   ├── Table_RQ9_2_component_ablation.{csv,tex}
│   ├── summary_rq9.json
│   ├── figure_captions.txt
│   └── README_RQ9.md
```

### Paths Relativos Verificados:

```python
BASE_DIR = Path('../..')                                    # ✅ Root del proyecto
DATA_DIR = BASE_DIR / 'data'                               # ✅ Datos BDD100K
TEMPERATURE_FILE = BASE_DIR / 'fase 4' / 'outputs' / ...   # ✅ Resultados Fase 4
coco_file = DATA_DIR / 'bdd100k_coco' / 'val_eval.json'   # ✅ Anotaciones
img_path = DATA_DIR / 'bdd100k' / 'bdd100k' / ...         # ✅ Imágenes
```

### Consistencia con Fases Anteriores:

- ✅ Usa misma estructura de hooks que RQ6
- ✅ Usa misma función `compute_iou()` que Fase 5
- ✅ Usa misma función `normalize_label()` que todas las fases
- ✅ Carga temperatura de Fase 4 igual que RQ7
- ✅ Formato de salida consistente con RQ6-RQ8

### Próximos Pasos:

1. **Ejecutar celda 1**: Configuración e imports (~30 segundos)
2. **Ejecutar celda 5**: Cargar modelo GroundingDINO (~30 segundos)
3. **Ejecutar celda 9**: Registrar hooks del decoder (~5 segundos)
4. **Ejecutar celda 11**: Procesar dataset **[MÁS COSTOSA: 30-60 min]**
5. **Ejecutar resto**: Métricas, figuras y tablas (~5 minutos)

**Total estimado**: 40-70 minutos con GPU